In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://www.fantasypros.com/nfl/cheatsheets/top-players.php?type=dynasty'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

players = []
position = []
team = []

pl = soup.find('div', class_='player-list')
for li in pl.find_all("li"):
    a_tag = li.find('a')
    if a_tag:
        players.append(a_tag.get('fp-player-name', a_tag.get_text(strip=True)))
    small_tag = li.find('small', class_='grey')
    if small_tag:
        pos_tm = small_tag.get_text(strip=True)
        pos, tm = pos_tm.split('-')
        position.append(pos)
        team.append(tm)
        
df = pd.DataFrame({
    'player': players,
    'position': position,
    'team': team
})

In [3]:
df

,player,position,team
0,Justin Jefferson,WR,MIN
1,CeeDee Lamb,WR,DAL
2,Ja'Marr Chase,WR,CIN
3,Malik Nabers,WR,NYG
4,Amon-Ra St. Brown,WR,DET
...,...,...,...
345,Michael Carter,RB,ARI
346,Hayden Hurst,TE,LAC
347,Kylen Granson,TE,IND
348,Jared Wiley,TE,KC


In [4]:
players[0].replace(" ", "-")

'Justin-Jefferson'

In [3]:
def cfb_eval(player):
    url = 'https://www.playerprofiler.com/nfl/PLAYER/'.replace("PLAYER",player)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    info = []
    for div in soup.find_all('div', class_='font-display leading-none')[4:]:
        info.append(div.text)
        
    metrics = []
    for div in soup.find_all('div', class_='text-sm font-display text-center leading-none')[:5]:
        metrics.append(div.text)
        
    peak = max(peakPPG(player)['ppg'])
    
    cfb = soup.find_all('table', class_='w-full')[-1]
    career = []
    szn = []
    i = 0
    for line in cfb.find_all('td'):
        if line:
            szn.append(line.get_text(strip=True).split("(")[0])
            if len(szn) == 10:
                szn = szn + info
                szn = szn + metrics
                szn.append(peak)
                szn.append(player.replace("-", " "))
                career.append(szn)
                szn = []
                i += 1
        
    career = pd.DataFrame(career, 
                         columns=['year', 'games', 'rec', 'yds', 'tgtsh','ctr','tds','st','age','school',"draft", 
                                  "college", "age", "college_dom", "college_tgts", "breakout_age", "40", "speed",
                                  "burst", 'agility', "catch", 'peakPPG', 'name'])
    
    return career.drop("college", axis=1)

In [53]:
cfb_eval('Justin-Jefferson')

,year,games,rec,yds,tgtsh,ctr,tds,st,age,school,...,college_dom,college_tgts,breakout_age,40,speed,burst,agility,catch,peakPPG,name
0,2017,2,0,0,-,-,0,0,18,LSU,...,30.0%,21.7%,19.6,4.43,104.9,126.8,,10.06,21.7,Justin Jefferson
1,2018,13,54,875,24.0%,58.1%,6,2,19,LSU,...,30.0%,21.7%,19.6,4.43,104.9,126.8,,10.06,21.7,Justin Jefferson
2,2019,15,111,1540,23.6%,82.8%,18,2,20,LSU,...,30.0%,21.7%,19.6,4.43,104.9,126.8,,10.06,21.7,Justin Jefferson


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [5]:
def peakPPG(player):
    #ppgDB = {}
    playerHist = {}
    ppg = []
    pos_rank = []

    options = Options()
    options.headless = True
    driver = webdriver.Chrome(service=Service('../chromedriver-mac-x64/chromedriver'), options=options)
    driver.get("https://www.playerprofiler.com/nfl/PLAYER/".replace('PLAYER',player))
    try:
        load_more = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.ID, 'moreSeasonStats'))
        )
        driver.execute_script("arguments[0].click();", load_more)
        time.sleep(5)
    except Exception as e:
        pass
    table = driver.find_element(By.ID,'seasonalStatsTable')
    for div in table.find_elements(By.TAG_NAME, "div"):
        match = re.match(r'([\d]+\.[\d]+)\s*\(#(\d+)\)', div.text)
        if match:
            ppg.append(float(match.group(1)))
            pos_rank.append(int(match.group(2)))
    driver.quit()
    playerHist['ppg'] = ppg
    playerHist['posRank'] = pos_rank 
    #ppgDB[player.replace('-', " ")] = playerHist
    #return pd.DataFrame(ppgDB)
    return playerHist

In [24]:
test = peakPPG("Randy-Moss")
max(test['ppg'])

24.1

In [23]:
test

,Randy Moss
posRank,"[89, 70, 4, 13, 1, 50, 21, 10, 1, 4, 6, 4]"
ppg,"[5.6, 7.5, 17.8, 14.5, 24.1, 8.9, 13.0, 16.8, ..."


In [58]:
(pd.DataFrame(test['Randy Moss']['posRank']) <= 12).sum().values[0]

7

In [59]:
(pd.DataFrame(test['Randy Moss']['posRank']) <= 24).sum().values[0]

9

In [58]:
result = cfb_eval(df[df['position'] == 'TE']['player'].iloc[0].replace(" ", "-"))
for player in df[df['position'] == 'TE']['player'][1:]:
    try:
        result = pd.concat([result, cfb_eval(player.replace(".","").replace("Jr","").replace(" ", "-").replace("'",""))])
    except Exception as e: 
        print(player)
    #print(player.replace(".","").replace("Jr","").replace(" ", "-").replace("'",""))
    

Isaiah Likely
Ben Sinnott
Chig Okonkwo
Jaheim Bell


In [60]:
result.to_csv('te.csv', index=False)

In [7]:
result = cfb_eval(df[df['position'] == 'WR']['player'].iloc[0].replace(" ", "-"))
for player in df[df['position'] == 'WR']['player'][1:]:
    try:
        result = pd.concat([result, cfb_eval(player.replace(".","").replace("Jr","").replace(" ", "-").replace("'",""))])
    except Exception as e: 
        print(player)

Marvin Harrison Jr.
Tyreek Hill
Tank Dell
Deebo Samuel Sr.
Cooper Kupp
Christian Watson
Ricky Pearsall
Roman Wilson
Joshua Palmer
Gabe Davis
Javon Baker
Adam Thielen
Devontez Walker
Malik Washington
Jacob Cowing
Brenden Rice
A.T. Perry
Jamari Thrash
Kendrick Bourne
Ainias Smith
John Metchie III


In [9]:
missed = ['Tyreek-Hill', 'Nathaniel-Dell', 'Deebo-Samuel', 
          'Cooper-Kupp', 'Christian-Watson', 'Josh-Palmer', 
          'Gabriel-Davis', 'Adam-Thielen', 'Kendrick-Bourne']
for player in missed:
    try:
        result = pd.concat([result, cfb_eval(player)])
    except Exception as e:
        print(player)
        print(f"Error: {e}")

Tyreek-Hill
Cooper-Kupp
Christian-Watson
Adam-Thielen
Kendrick-Bourne


In [13]:
result.to_csv('wr.csv', index=False)

In [15]:
result = cfb_eval(df[df['position'] == 'QB']['player'].iloc[0].replace(" ", "-"))
for player in df[df['position'] == 'QB']['player'][1:]:
    try:
        result = pd.concat([result, cfb_eval(player.replace(".","").replace("Jr","").replace(" ", "-").replace("'",""))])
    except Exception as e: 
        print(player)
        print(f"Error: {e}")

Patrick Mahomes II
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="seasonalStatsTable"]"}
  (Session info: chrome=129.0.6668.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000105d3fd08 chromedriver + 4996360
1   chromedriver                        0x0000000105d375ca chromedriver + 4961738
2   chromedriver                        0x00000001058dad10 chromedriver + 388368
3   chromedriver                        0x000000010592730f chromedriver + 701199
4   chromedriver                        0x00000001059273f1 chromedriver + 701425
5   chromedriver                        0x000000010596c464 chromedriver + 984164
6   chromedriver                        0x000000010594b9dd chromedriver + 850397
7   chromedriver                        0x0000000105969a00 chromedriver + 973312
8   chrom

In [21]:
result[result['name']=='Andy Dalton'].loc[0]

year                   2007
games                    13
rec                     371
yds                    2459
tgtsh                   6.6
ctr                   59.8%
tds                      10
st                      232
age                      20
school                  TCU
draft                  2.03
age                    36.9
college_dom            81.2
college_tgts            9.0
breakout_age           20.9
40                     4.87
speed                 104.6
burst                 11.20
agility                  56
catch                    29
peakPPG                19.3
name            Andy Dalton
Name: 0, dtype: object

In [22]:
cols = ['year', 'games', 'pass_att', 'yds', 'ypa', 'comp%', 'pass_tds', 
 'rush_yds', 'age', 'school', 'draft', 'age', 'qbr', 
 'college_ypa', 'breakout_age', '40', 'burst', 'agility', 
 'throw_velo', 'wonderlic', 'peakPPG', 'name']
result.columns = cols

In [23]:
result

,year,games,pass_att,yds,ypa,comp%,pass_tds,rush_yds,age,school,...,qbr,college_ypa,breakout_age,40,burst,agility,throw_velo,wonderlic,peakPPG,name
0,2015,2,6,51,8.5,66.7%,0,40,19,Wyoming,...,63.2,7.7,20.3,4.75,118.1,11.30,62,37,25.4,Josh Allen
1,2016,14,373,3203,8.6,56.0%,28,523,20,Wyoming,...,63.2,7.7,20.3,4.75,118.1,11.30,62,37,25.4,Josh Allen
2,2017,11,270,1812,6.7,56.3%,16,204,21,Wyoming,...,63.2,7.7,20.3,4.75,118.1,11.30,62,37,25.4,Josh Allen
0,2015,12,247,1840,7.5,54.7%,12,960,18,Louisville,...,85.0,8.5,18.7,4.44,,,49,13,28.2,Lamar Jackson
1,2016,13,409,3543,8.7,56.2%,30,1571,19,Louisville,...,85.0,8.5,18.7,4.44,,,49,13,28.2,Lamar Jackson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2015,13,395,2662,6.7,60.0%,20,370,22,NC State,...,68.2,6.9,21.7,4.94,110.8,11.70,56,24,14.7,Jacoby Brissett
0,2007,13,371,2459,6.6,59.8%,10,232,20,TCU,...,81.2,9.0,20.9,4.87,104.6,11.20,56,29,19.3,Andy Dalton
1,2008,11,307,2242,7.3,59.3%,11,432,21,TCU,...,81.2,9.0,20.9,4.87,104.6,11.20,56,29,19.3,Andy Dalton
2,2009,13,323,2756,8.5,61.6%,23,512,22,TCU,...,81.2,9.0,20.9,4.87,104.6,11.20,56,29,19.3,Andy Dalton


In [24]:
result.to_csv('qb.csv', index=True)

In [7]:
result = cfb_eval(df[df['position'] == 'RB']['player'].iloc[0].replace(" ", "-"))
for player in df[df['position'] == 'RB']['player'][1:]:
    try:
        result = pd.concat([result, cfb_eval(player.replace(".","").replace("Jr","").replace(" ", "-").replace("'",""))])
    except Exception as e: 
        print(player)
        print(f"Error: {e}")

Kenneth Walker III
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="seasonalStatsTable"]"}
  (Session info: chrome=129.0.6668.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010bf8ad08 chromedriver + 4996360
1   chromedriver                        0x000000010bf825ca chromedriver + 4961738
2   chromedriver                        0x000000010bb25d10 chromedriver + 388368
3   chromedriver                        0x000000010bb7230f chromedriver + 701199
4   chromedriver                        0x000000010bb723f1 chromedriver + 701425
5   chromedriver                        0x000000010bbb7464 chromedriver + 984164
6   chromedriver                        0x000000010bb969dd chromedriver + 850397
7   chromedriver                        0x000000010bbb4a00 chromedriver + 973312
8   chrom

Trey Sermon
Error: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=129.0.6668.101)
Stacktrace:
0   chromedriver                        0x000000010f81bd08 chromedriver + 4996360
1   chromedriver                        0x000000010f8135ca chromedriver + 4961738
2   chromedriver                        0x000000010f3b6d10 chromedriver + 388368
3   chromedriver                        0x000000010f39e3e6 chromedriver + 287718
4   chromedriver                        0x000000010f39e2e3 chromedriver + 287459
5   chromedriver                        0x000000010f3b9082 chromedriver + 397442
6   chromedriver                        0x000000010f445423 chromedriver + 971811
7   chromedriver                        0x000000010f427753 chromedriver + 849747
8   chromedriver                        0x000000010f3f6635 chromedriver + 648757
9   chromedriver                        0x000000010f3f6e5e chromedr

In [9]:
missed = ['Kenneth-Walker', 'Trey-Sermon']
for player in missed:
    try:
        result = pd.concat([result, cfb_eval(player)])
    except Exception as e:
        print(player)
        print(f"Error: {e}")

In [10]:
result

,year,games,rec,yds,tgtsh,ctr,tds,st,age,school,...,college_dom,college_tgts,breakout_age,40,speed,burst,agility,catch,peakPPG,name
0,2020,9,86,703,6,15,196,6.1%,18,Texas,...,36.8%,6.1,10.0%,4.46,108.7,124.9,,,15.5,Bijan Robinson
1,2021,10,195,1127,15,26,295,9.6%,19,Texas,...,36.8%,6.1,10.0%,4.46,108.7,124.9,,,15.5,Bijan Robinson
2,2022,12,258,1580,20,19,314,7.9%,20,Texas,...,36.8%,6.1,10.0%,4.46,108.7,124.9,,,15.5,Bijan Robinson
0,2019,12,186,897,10,23,252,5.8%,18,Iowa State,...,43.9%,5.8,10.7%,4.39,116.9,131.0,,,17.1,Breece Hall
1,2020,12,279,1,23,23,180,7.2%,19,Iowa State,...,43.9%,5.8,10.7%,4.39,116.9,131.0,,,17.1,Breece Hall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,2021,12,263,1636,19,13,89,5.0%,21,Michigan State,...,50.2%,5.5,5.4%,4.38,114.7,118.8,,,22.0,Kenneth Walker
0,2017,13,121,744,7,16,139,-,18,Oklahoma,...,16.5%,7.5,8.4%,4.66,91.2,125.4,11.14,,5.2,Trey Sermon
1,2018,14,164,947,13,12,181,4.0%,19,Oklahoma,...,16.5%,7.5,8.4%,4.66,91.2,125.4,11.14,,5.2,Trey Sermon
2,2019,10,54,385,5,8,71,3.2%,20,Oklahoma,...,16.5%,7.5,8.4%,4.66,91.2,125.4,11.14,,5.2,Trey Sermon


In [11]:
result[result['name']=='Breece Hall'].loc[0]

year                   2019
games                    12
rec                     186
yds                     897
tgtsh                    10
ctr                      23
tds                     252
st                     5.8%
age                      18
school           Iowa State
draft                  2.04
age                    23.3
college_dom           43.9%
college_tgts            5.8
breakout_age          10.7%
40                     4.39
speed                 116.9
burst                 131.0
agility                    
catch                      
peakPPG                17.1
name            Breece Hall
Name: 0, dtype: object

In [12]:
cols = ['year', 'games', 'atts', 'rush_yds', 'tds', 'rec', 
        'rec_yds', 'tgt_share', 'age', 'school', 'draft', 'age',
       'college_dom', 'college_ypc', 'college_tgt_share',
       '40', 'speed', 'burst', 'agility', 'bench', 'peakPPG', 'name']
result.columns = cols

In [15]:
result

,year,games,atts,rush_yds,tds,rec,rec_yds,tgt_share,age,school,...,college_dom,college_ypc,college_tgt_share,40,speed,burst,agility,bench,peakPPG,name
0,2020,9,86,703,6,15,196,6.1%,18,Texas,...,36.8%,6.1,10.0%,4.46,108.7,124.9,,,15.5,Bijan Robinson
1,2021,10,195,1127,15,26,295,9.6%,19,Texas,...,36.8%,6.1,10.0%,4.46,108.7,124.9,,,15.5,Bijan Robinson
2,2022,12,258,1580,20,19,314,7.9%,20,Texas,...,36.8%,6.1,10.0%,4.46,108.7,124.9,,,15.5,Bijan Robinson
0,2019,12,186,897,10,23,252,5.8%,18,Iowa State,...,43.9%,5.8,10.7%,4.39,116.9,131.0,,,17.1,Breece Hall
1,2020,12,279,1,23,23,180,7.2%,19,Iowa State,...,43.9%,5.8,10.7%,4.39,116.9,131.0,,,17.1,Breece Hall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,2021,12,263,1636,19,13,89,5.0%,21,Michigan State,...,50.2%,5.5,5.4%,4.38,114.7,118.8,,,22.0,Kenneth Walker
0,2017,13,121,744,7,16,139,-,18,Oklahoma,...,16.5%,7.5,8.4%,4.66,91.2,125.4,11.14,,5.2,Trey Sermon
1,2018,14,164,947,13,12,181,4.0%,19,Oklahoma,...,16.5%,7.5,8.4%,4.66,91.2,125.4,11.14,,5.2,Trey Sermon
2,2019,10,54,385,5,8,71,3.2%,20,Oklahoma,...,16.5%,7.5,8.4%,4.66,91.2,125.4,11.14,,5.2,Trey Sermon


In [16]:
result.to_csv('rb.csv', index=True)

In [13]:
hofte = pd.read_csv('hofte.csv', header=1)

In [9]:
url = 'https://www.playerprofiler.com/position/tight-end/'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
tes = []
for h2 in soup.find_all('h2'):
    tes.append(h2.text)

In [10]:
result = cfb_eval(tes[0].replace(" ", "-"))
for player in tes[1:]:
    try:
        result = pd.concat([result, cfb_eval(player.replace(".","").replace("Jr","").replace(" ", "-").replace("'",""))])
    except Exception as e: 
        print(player)
        print(f"Error: {e}")

Erick All
Error: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.70)
Stacktrace:
0   chromedriver                        0x0000000106ad3d08 chromedriver + 4996360
1   chromedriver                        0x0000000106acb5ca chromedriver + 4961738
2   chromedriver                        0x000000010666ed10 chromedriver + 388368
3   chromedriver                        0x000000010665779a chromedriver + 292762
4   chromedriver                        0x00000001066574ba chromedriver + 292026
5   chromedriver                        0x00000001066565d9 chromedriver + 288217
6   chromedriver                        0x000000010667ab81 chromedriver + 437121
7   chromedriver                        0x00000001066fdd41 chromedriver + 974145
8   chromedriver                        0x00000001066df753 chromedriver + 849747
9   chromedriver                        0x00000001066ae6

Nate Adkins
Error: 23 columns passed, passed data had 22 columns
Isaac Rex
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="seasonalStatsTable"]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000109757d08 chromedriver + 4996360
1   chromedriver                        0x000000010974f5ca chromedriver + 4961738
2   chromedriver                        0x00000001092f2d10 chromedriver + 388368
3   chromedriver                        0x000000010933f30f chromedriver + 701199
4   chromedriver                        0x000000010933f3f1 chromedriver + 701425
5   chromedriver                        0x0000000109384464 chromedriver + 984164
6   chromedriver                        0x00000001093639dd chromedriver + 850397
7   chromedriver                   

David Grinnage
Error: max() arg is an empty sequence
Mark Harrison
Error: max() arg is an empty sequence
Jaeden Graham
Error: 23 columns passed, passed data had 22 columns
Nick Keizer
Error: 23 columns passed, passed data had 22 columns
Zach Davidson
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="seasonalStatsTable"]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000103ac0d08 chromedriver + 4996360
1   chromedriver                        0x0000000103ab85ca chromedriver + 4961738
2   chromedriver                        0x000000010365bd10 chromedriver + 388368
3   chromedriver                        0x00000001036a830f chromedriver + 701199
4   chromedriver                        0x00000001036a83f1 chromedriver + 701425
5   chromedriver     

Jason Vander Laan
Error: max() arg is an empty sequence
Ian Bunting
Error: max() arg is an empty sequence
Nate Becker
Error: max() arg is an empty sequence
Daniel Helm
Error: max() arg is an empty sequence
Artayvious Lynn
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="seasonalStatsTable"]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000107047d08 chromedriver + 4996360
1   chromedriver                        0x000000010703f5ca chromedriver + 4961738
2   chromedriver                        0x0000000106be2d10 chromedriver + 388368
3   chromedriver                        0x0000000106c2f30f chromedriver + 701199
4   chromedriver                        0x0000000106c2f3f1 chromedriver + 701425
5   chromedriver                        0x00000001

Brandon Myers
Error: 23 columns passed, passed data had 21 columns
James Hanna
Error: 23 columns passed, passed data had 22 columns
Davis Koppenhaver
Error: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=130.0.6723.70)
Stacktrace:
0   chromedriver                        0x0000000107114d08 chromedriver + 4996360
1   chromedriver                        0x000000010710c5ca chromedriver + 4961738
2   chromedriver                        0x0000000106cafd10 chromedriver + 388368
3   chromedriver                        0x0000000106c973e6 chromedriver + 287718
4   chromedriver                        0x0000000106c972e3 chromedriver + 287459
5   chromedriver                        0x0000000106cb2082 chromedriver + 397442
6   chromedriver                        0x0000000106d3e423 chromedriver + 971811
7   chromedriver                        0x0000000106d20753 chromedriver + 849747
8   chromed

Brandon Williams
Error: 23 columns passed, passed data had 21 columns
Zach Miller
Error: 23 columns passed, passed data had 21 columns
Mychal Rivera
Error: 23 columns passed, passed data had 22 columns
Dominique Dafney
Error: max() arg is an empty sequence
Tony Poljan
Error: max() arg is an empty sequence
Asa Watson
Error: max() arg is an empty sequence
Asante Cleveland
Error: max() arg is an empty sequence
Brandon Pettigrew
Error: 23 columns passed, passed data had 22 columns
Josh Pederson
Error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="seasonalStatsTable"]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010b5ffd08 chromedriver + 4996360
1   chromedriver                        0x000000010b5f75ca chromedriver + 4961738
2   chromedriver   

KeyboardInterrupt: 

In [12]:
result.to_csv('atte.csv', index=True)

In [13]:
cfb_eval("Rob-Gronkowski")

ValueError: 23 columns passed, passed data had 22 columns